## Final Project Submission

Please fill out:
* Student name: Esther Terry Munene
* Student pace: full time
* Scheduled project review date/time: march 22nd 2024
* Instructor name: MaryAnne Mwikali
* Blog post URL: N/A


 1.Introduction
 
 Overview of the project
 

In [2]:
# Your code here - remember to use markdown cells for comments as well!